In [6]:
import torch
import csv
import os
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
from dataset_reader import Traces_Dataset
# from mlp_model import MLP
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import uuid

import torch.nn.init as init

In [2]:
# initialize a dictionary of training history to store in a csv file
history_dict = {}

unique_id = str(uuid.uuid4())[:8]
history_dict['unique_id'] = unique_id
print(f'Experiment ID: {unique_id}')

# making training reproducible
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
history_dict['seed'] = seed

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Uncomment this line
print(f"Using device: {device}")

Experiment ID: d6d3e3f7
Using device: cuda


In [3]:
# load and process dataset 
dataset = Traces_Dataset('dataset2mil.csv')
# dataset = Traces_Dataset('dataset_test.csv')
dataset.split_dataset(0.95, 0.05, 0)
dataset.clean_features()
dataset.find_mean_std()
dataset.normalize()
print(dataset.inputs.shape)
history_dict['normalize_mean'] = dataset.train_mean.tolist()
history_dict['normalize_std'] = dataset.train_std.tolist()
history_dict['dataset'] = (dataset.inputs.shape[0], dataset.inputs.shape[1])

# initialize train, val, test set
X_train = dataset[dataset.train_set.indices][0]
Y_train = dataset[dataset.train_set.indices][1]

X_val = dataset[dataset.val_set.indices][0]
Y_val = dataset[dataset.val_set.indices][1]

X_test = dataset[dataset.test_set.indices][0]
Y_test = dataset[dataset.test_set.indices][1]


torch.Size([2000000, 321])


In [74]:
class MLP(nn.Module): 
    def __init__(self, n_features, n_params=7):
        super().__init__()
        self.layers = nn.Sequential(
                nn.Linear(n_features, 125),
                nn.BatchNorm1d(125),
                nn.SiLU(),
                nn.Linear(125, 125),
                nn.BatchNorm1d(125),
                nn.SiLU(),
                nn.Linear(125, 125),
                nn.BatchNorm1d(125),
                nn.SiLU(),
                nn.Linear(125, 125),
                nn.BatchNorm1d(125),
                nn.SiLU(),
                nn.Linear(125, n_params)
            )
        
    def forward(self, x):
        return self.layers(x)
    
    def initialize_weights(self):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                # Apply Xavier initialization to linear layers
                init.xavier_uniform_(layer.weight)
                # You can also initialize biases, for example, with zeros
                if layer.bias is not None:
                    init.constant_(layer.bias, 0)

In [75]:
# initialize NN model
model = MLP(dataset.inputs.shape[1], dataset.params.shape[1]).to(device)
model.initialize_weights()

# store model architecture
architecture = []
for name, layer in model.named_children():
    architecture.append(layer)
history_dict['architecture'] = architecture

In [76]:
print(model)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=321, out_features=125, bias=True)
    (1): BatchNorm1d(125, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU()
    (3): Linear(in_features=125, out_features=125, bias=True)
    (4): BatchNorm1d(125, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): SiLU()
    (6): Linear(in_features=125, out_features=125, bias=True)
    (7): BatchNorm1d(125, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): SiLU()
    (9): Linear(in_features=125, out_features=125, bias=True)
    (10): BatchNorm1d(125, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): SiLU()
    (12): Linear(in_features=125, out_features=7, bias=True)
  )
)


In [77]:
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001) 

# record optimizer
lr = optimizer.param_groups[0]['lr']
history_dict['optimizer'] = f'{optimizer.__class__.__name__} {lr}'

In [78]:
# functions to save and load best nn model, and create an unique id to store the model
def checkpoint(model, filename, folder='models'):
    os.makedirs(folder, exist_ok=True)
    filepath = os.path.join(folder, filename)
    torch.save(model.state_dict(), filepath)
    return filepath
def resume(model, filepath):
    model.load_state_dict(torch.load(filepath))

# this is a 1d vector containing all weight in the nn model, no bias included. 
def weights_1d(model): 
    # Get parameters with gradients
    parameters_with_grad = [param for param in model.parameters() if param.requires_grad]
    # Concatenate parameters into a 1D tensor
    flat_parameters = torch.cat([param.view(-1) for param in parameters_with_grad])
    return flat_parameters
print(f'Number of model parameters: {weights_1d(model).shape[0]}')
weights_change_updates = np.array([])  # using numpy array to save memory 
weights_change_epochs = []


Number of model parameters: 89382


In [79]:
# training parameters
n_epochs = 300   # number of epochs to run
batch_size = 1024  # size of each batch
history_dict['epochs'] = n_epochs
history_dict['batch size'] = batch_size

# initialize dataloader 
train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, Y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# initialization train, val losses
train_losses = []
val_losses = []

best_validation_loss = float('inf')

# record the weights with no training
# with torch.no_grad():
#     previous_weights_epoch = weights_1d(model)

In [80]:
# Training loop
for epoch in range(1, n_epochs + 1):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = loss_fn(outputs, labels)  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        # record the weights changes across each updates (batches)
        # with torch.no_grad():
        #     current_weights = weights_1d(model)
        #     if 'previous_weights' in locals():
        #         weight_change = torch.norm(current_weights - previous_weights).item()#.cpu()
        #         weights_change_updates = np.append(weights_change_updates, weight_change)
        #     previous_weights = current_weights

        total_loss += loss.item()

    # Average training loss for the epoch
    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0.0

    with torch.no_grad():  # Disable gradient calculation during validation
        # record the weights change across epochs
        # current_weights_epoch = weights_1d(model)
        # if 'previous_weights_epoch' in locals():
        #     weight_change_epoch = torch.norm(current_weights_epoch - previous_weights_epoch).item()#.cpu()
        #     weights_change_epochs.append(weight_change_epoch)
        # previous_weights_epoch = current_weights_epoch

        # validation
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            val_loss = loss_fn(val_outputs, val_labels)
            total_val_loss += val_loss.item()

    # Average validation loss for the epoch
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f'{epoch}: train-{avg_train_loss}, val-{avg_val_loss}')

    # saving the model with best training mse
    # if len(train_losses) != 1:
    #     if avg_train_loss < train_losses[-2]: 
    #         best_loss = avg_train_loss
    #         best_epoch = epoch
    #         model_path = checkpoint(model, f"model_{unique_id}.pth")
    if avg_val_loss < best_validation_loss:
        best_epoch = epoch
        model_path = checkpoint(model, f"model_{unique_id}.pth")
        best_training_loss = avg_train_loss
        best_validation_loss = avg_val_loss


1: train-579.0567193298505, val-19.596047712832082
2: train-13.89483207104535, val-9.642817039879
3: train-9.45454570676746, val-10.227993147713798
4: train-7.82828332894835, val-9.791195781863465
5: train-6.736649763995204, val-9.405744795896569
6: train-6.051027098349456, val-10.758202689034599
7: train-5.423118731328126, val-5.832976136888776
8: train-5.016837889906661, val-9.267020303375867
9: train-4.63913432662857, val-8.532382673146774
10: train-4.267233007170003, val-6.088935229243065
11: train-4.0087227920262976, val-4.162988711376579
12: train-3.8057663104143638, val-10.483747131970464
13: train-3.6236545795510553, val-6.080276902841062
14: train-3.3985254903291833, val-3.580239792259372
15: train-3.272798581883825, val-4.756282694485723
16: train-3.177991761408489, val-14.982386044093541
17: train-3.0592096365089048, val-3.773691089785829
18: train-2.9238000280255902, val-3.206713077973346
19: train-2.8361674919082174, val-20.06652561499148
20: train-2.801461866382381, val-6

KeyboardInterrupt: 

In [ ]:
# record training, validationg losses, weight updates, and the result model path
history_dict['best_epoch'] = best_epoch
history_dict['best_val'] = best_validation_loss
history_dict['best_train'] = best_training_loss
history_dict['training'] = train_losses
history_dict['validation'] = val_losses
history_dict['weights'] = weights_change_epochs
history_dict['model'] = model_path


In [ ]:
def experiment_records(row_data, file_path = 'experiment records.csv'): 
    '''
    row_data is a dictionary of the row_data we want to store
    '''
    # Check if the file exists
    file_exists = os.path.isfile(file_path)

    with open(file_path, mode='a' if file_exists else 'w', newline='') as csv_file:
        # Create a CSV writer object
        csv_writer = csv.writer(csv_file)

        # If the file is newly created, write the header row
        if not file_exists:
            header_row = row_data.keys() if isinstance(row_data, dict) else row_data
            csv_writer.writerow(header_row)

        # Write the data row
        if isinstance(row_data, dict):
            csv_writer.writerow(row_data.values())
        else:
            csv_writer.writerow(row_data)


print(f"Best validation loss: {best_validation_loss}, at epoch: {best_epoch}")
#print("RMSE: %.2f" % np.sqrt(best_mse))
print('Number of total training samples: ', X_train.shape[0])

experiment_records(history_dict)